Intro to minikanren

TOC:
*   implement microkanren
*   write minikanren api through macros

In [1]:
import itertools as it

A $\mu$Kanren program proceeds through the application of a **goal** to a **state**. Goals are often understood by analogy to predicates. Whereas the application of a predicate to an element of its domain can be either true or false, a goal pursued in a given state can either succeed or fail.

A **state** is a pair of a substitution (represented as a dictionary) and a non-negative integer representing a fresh- variable counter.

LogicVariables are ints, until I want to distinguish them from ints. In that case, I check is_logic_var

In [2]:
empty_state = ({}, 0)

def is_logic_var(x):
    return isinstance(x, LogicVariable)

class LogicVariable(object):
    """
    An alias to an int
    """
    def __init__(self, i):
        self.i = i
        
    def __cmp__(self, i):
        return cmp(self.i, rhs.i)
    
    def __eq__(self, rhs):
        return is_logic_var(rhs) and self.i == rhs.i
    
    def __hash__(self):
        return hash(self.i)

    def __repr__(self):
        return "LV(%d)" % self.i
    


The **walk** operator searches for a variable's value in the substitution; the *ext_s* operator extends the substitution with a new binding. When a non-variable term is walked, the term itself is returned. When extending the substitution, the 􏰀first argument is always a variable, and the second is an arbitrary term.

In [3]:
def walk(var, substitutions):
    while is_logic_var(var) and var in substitutions:
        var = substitutions[var]
    return var

def ext_s(var, value, substitutions):
    s = dict(substitutions)
    s[var] = value
    return s

In [5]:
LV = LogicVariable
assert walk(LV(0), {}) == LV(0)
assert walk(LV(0), {LV(0): 1}) == 1
assert walk(LV(0), {LV(0): LV(1), LV(1): "something else"}) == "something else"

Explain unify

In [80]:
import collections

def is_sequence(o):
    """
    I want it to fail on strings
    """
    return hasattr(o, '__iter__')

def unify_sequences(xs, ys, substitutions):
    if len(xs) != len(ys):
        return None
    for a, b in it.izip(xs, ys):
        substitutions = unify(a, b, substitutions) 
    return substitutions

def unify(x, y, substitutions):
    if substitutions is None:
        return None

    x = walk(x, substitutions)
    y = walk(y, substitutions)

    if x == y:
        return substitutions
    elif is_logic_var(x):
        return ext_s(x, y, substitutions)
    elif is_logic_var(y):
        return ext_s(y, x, substitutions)
    elif is_sequence(x) and is_sequence(y):
        return unify_sequences(x, y, substitutions)
    return None

In [81]:
assert unify(LV(0), LV(1), {}) == {LV(0) : LV(1)} #  0 is 1
assert unify(LV(0), LV(1), {LV(0): "x", LV(1): "y"}) is None  #  they do not unify
assert unify(LV(0), 5, {}) == {LV(0): 5}
assert unify((LV(0), 1, LV(1)), (1, 2, 3), {}) is None # the second elements differ
assert unify((LV(0), 1, LV(1)), (1, 1, 3), {}) == {LV(0): 1, LV(1): 3} # assign the first and the last one

## Goals builder


1.   `equiv` builds a goal which succeeds if its two arguments unify, i.e. it yields the substitutions which make its arguments unify

In [82]:
def equiv(x, y):
    def _goal(state):
        substitutions, counter = state
        unified = unify(x, y, substitutions)
        if unified is not None:
            yield (unified, counter)
    return _goal

In [83]:
assert list(equiv(LV(0), 5)(empty_state)) == [({LV(0): 5}, 0)]
assert list(equiv(LV(0), LV(1))(empty_state)) == [({LV(0): LV(1)}, 0)]

The call/fresh goal constructor takes a unary function f whose body is a goal, and itself returns a goal. This returned goal, when provided a state s/c, binds the formal parameter of f to a new logic variable (built with the variable construc- tor operator var), and passes a state, with the substitution it originally received and a newly incremented fresh-variable counter, c, to the goal that is the body of f.

In [84]:
def call_fresh(goal):
    def _new_goal(state):
        substitutions, counter = state
        return goal(LogicVariable(counter))((substitutions, counter+1))
    return _new_goal


In [85]:
from functools import partial as p

is_five = p(equiv, 5)
assert list(call_fresh(is_five)(empty_state)) == [({LV(0): 5}, 1)]

In [123]:
def disj(*goals):
    """
    it returns a goal which succeeds if either of the arguments succeeds
    """
    def _new_goal(state):
        return it.chain.from_iterable(g(state) for g in goals)
    return _new_goal

def make_stream(goal, s):
    return it.chain.from_iterable(it.imap(goal, s))

def conj(*goals):
    """
    It returns a goal which succeeds if all the goals passed as argument succeed
    for that state.
    """
    def _new_goal(state):
        stream = goals[0](state)
        for g in goals[1:]:
            stream = make_stream(g, stream)
        return stream
    return _new_goal

ANY = disj
ALL = conj

Minikanren utility

fresh

In [124]:
def f(x):
    def _g(y):
        return equiv(x, y)
    return call_fresh(_g)

list(call_fresh(f)(empty_state))

[({LV(0): LV(1)}, 2)]

In [88]:
def g(x, y):
    return ANY(equiv(x, 2), equiv(y, 3), equiv(x, y))

list(call_fresh(lambda x: call_fresh(lambda y: g(x, y)))(empty_state))

[({LV(0): 2}, 2), ({LV(1): 3}, 2), ({LV(0): LV(1)}, 2)]

In [89]:
import inspect

def fresh_helper(curried_goal, nargs):
    if nargs <= 1:
        return call_fresh(curried_goal)
    else:
        return call_fresh(lambda x: fresh_helper(p(curried_goal, x), nargs-1))
    
def fresh(variadic_goal):
    positional_args = inspect.getargspec(variadic_goal).args
    return fresh_helper(variadic_goal, len(positional_args))
    
    
            
print list(fresh(f)(empty_state))
print list(fresh(g)(empty_state))

[({LV(0): LV(1)}, 2)]
[({LV(0): 2}, 2), ({LV(1): 3}, 2), ({LV(0): LV(1)}, 2)]


In [90]:
def run(goals):
    for s, c in fresh(goals)(empty_state):
        yield s
        
list(run(g))

[{LV(0): 2}, {LV(1): 3}, {LV(0): LV(1)}]

In [91]:
def reify_sequence(names, subs, objects):
    return [reify_s(names, subs, o) for o in objects]

def reify_s(names, subs, o):
    o = walk(o, subs)
    if is_logic_var(o):
        return "#_%s" % names.get(o, o).i
    elif is_sequence(o):
        return reify_sequence(names, subs, o)
    else:
        return o
    
def reify(substitutions, names):
    r = lambda var: reify_s(names, substitutions, LogicVariable(var))
    reified = {k: r(v) for k,v in names.iteritems()}
    return reified

def run(goal, stop=None):
    args = inspect.getargspec(goal).args
    names = {name: n for (n, name) in enumerate(args)}
    substitutions = fresh(goal)(empty_state)
    for s, c in it.islice(substitutions, stop):
        yield reify(s, names)

list(run(g))

[{'x': 2, 'y': '#_1'}, {'x': '#_0', 'y': 3}, {'x': '#_1', 'y': '#_1'}]

Examples
Querying Game of Thrones

In [92]:
got_characters = {0: ('catelyn', 'tully'),
                  1: ('eddard', 'stark'),
                  2: ('sansa', 'stark'),
                  3: ('benjen', 'stark'),
                  4: ('robb', 'stark'),
                  5: ('joffrey', 'baratheon'),
                  6: ('stannis', 'baratheon'),
                  7: ('cersei', 'lannister'),
                  8: ('tyrion', 'lannister'),
                  9: ('tommen', 'baratheon'),
                  10: ('jon', 'snow'),
                  11: ('myrcella', 'baratheon'),
                  12: ('tywin', 'lannister'),
                  13: ('jaime', 'lannister'),
                  14: ('rickon', 'stark'),
                  15: ('arya', 'stark'),
                  16: ('brandon', 'stark'),
                  17: ('renly', 'baratheon'),
                  18: ('robert', 'baratheon')}

In [93]:
def charactero(characterid, name, surname):
    return ANY(
        *[ALL(equiv(characterid, k), equiv(name, n), equiv(surname, s))
          for (k, (n, s)) in got_characters.iteritems()]
    )

In [94]:
list(run(
        lambda x, y: charactero(1, x, y)
    ))

[{'x': 'eddard', 'y': 'stark'}]

In [95]:
list(run(
        lambda x, y: charactero(x, y, 'baratheon')
    ))

[{'x': 5, 'y': 'joffrey'},
 {'x': 6, 'y': 'stannis'},
 {'x': 9, 'y': 'tommen'},
 {'x': 11, 'y': 'myrcella'},
 {'x': 17, 'y': 'renly'},
 {'x': 18, 'y': 'robert'}]

In [96]:
got_houses = {'stark': [0, 1, 2, 3, 4, 10,  15, 16],
              'tully': [0],
              'lannister': [5, 7, 8, 9, 11, 12, 13],
              'baratheon': [5, 7, 9, 11, 6, 18]}

def _houseo(house_name, characterid):
    g = ANY(
        *[ALL(equiv(house_name, k), equiv(characterid, v))
          for (k, vs) in got_houses.iteritems() for v in vs ])
    return g

In [97]:
list(run(
        lambda i: _houseo('baratheon', i)
    ))

[{'i': 5}, {'i': 7}, {'i': 9}, {'i': 11}, {'i': 6}, {'i': 18}]

In [98]:
def houseo(family_name, name, surname):
    def _f(characterid):
        return ALL(
            _houseo(family_name, characterid),
            charactero(characterid, name, surname)
        )
    return fresh(_f)

In [125]:
list(run(
        lambda house, surname: houseo(house, 'joffrey', surname)
    ))

[{'house': 'baratheon', 'surname': 'baratheon'},
 {'house': 'lannister', 'surname': 'baratheon'}]

In [126]:
list(run(
        lambda house, name: houseo(house, name, house)
    ))

# no jon snow

[{'house': 'stark', 'name': 'jon'}]

In [101]:
def not_equiv(x, y):
    """
    It succeeds when you it cannot unify x and y
    """
    def _goal(state):
        substitutions, counter = state
        unified = unify(x, y, substitutions)
        if unified is None:
            yield (substitutions, counter)
    return _goal
    
list(run(
        lambda house, name, family: ALL(
            houseo(house, name, family),
            not_equiv(house, family))
    ))

[{'family': 'lannister', 'house': 'baratheon', 'name': 'cersei'},
 {'family': 'baratheon', 'house': 'lannister', 'name': 'joffrey'},
 {'family': 'baratheon', 'house': 'lannister', 'name': 'tommen'},
 {'family': 'baratheon', 'house': 'lannister', 'name': 'myrcella'},
 {'family': 'tully', 'house': 'stark', 'name': 'catelyn'},
 {'family': 'snow', 'house': 'stark', 'name': 'jon'}]

Data structures

cons

In [102]:
def emptyo(d):
    return equiv(d, ())

def conso(a, b, cons):
    return equiv((a, b), cons)

def firsto(cons, elt):
    return fresh(lambda t: conso(elt, t, cons))

def tailo(cons, tail):
    return fresh(lambda first: conso(first, tail, cons))

In [103]:
list(run(
        lambda x: firsto(x, 1)))

[{'x': [1, '#_1']}]

In [104]:
def consify(lst):
    cons = ()
    for i in reversed(lst):
        cons = (i, cons)
    return cons
        
ten = consify(range(10))
list(run(
        lambda f, t, other: ALL(
            firsto(ten, f),
            tailo(ten, t),
            firsto(other, f))
    ))

[{'f': 0,
  'other': [0, '#_5'],
  't': [1, [2, [3, [4, [5, [6, [7, [8, [9, []]]]]]]]]]}]

Intermezzo: More minikanren

In [105]:
def conde(*disjs):
    """
    Syntactic sugar around the ANY - ALL template
    
    conde is also a form of if-then-else construct
    conde(
        [if-clause1 then],
        [if-clause2 then],
    )
    """
    return ANY(
        *[ALL(*conjs) for conjs in disjs]
    )

In [106]:
def membero(lst, elt):
    def _f(tail):
        return conde(
            [firsto(lst, elt)],
            [tailo(lst, tail), membero(tail, elt)])
    return fresh(_f)

In [107]:
list(run(
        lambda x: membero(consify(range(12)), 30)
        ))

[]

In [108]:
list(run(
        lambda x: membero(consify(range(12)), 11)
        ))

[{'x': '#_0'}]

In [109]:
list(run(
        lambda x: membero(consify(range(0)), x),
        stop=50))

[]

In [110]:
list(run(
        lambda x: membero(x, 10),
        stop=5))

[{'x': [10, '#_2']},
 {'x': ['#_2', [10, '#_4']]},
 {'x': ['#_2', ['#_4', [10, '#_6']]]},
 {'x': ['#_2', ['#_4', ['#_6', [10, '#_8']]]]},
 {'x': ['#_2', ['#_4', ['#_6', ['#_8', [10, '#_10']]]]]}]

In [111]:
def appendo(x, y, res):
    """
    concatenate x and y
    """
    def _f(a, b, c):
        return conde(
            # if x is empty, then res is y
            [emptyo(x), equiv(y, res)],
            # otherwise is something like cons(a, b), thus res will be cons(a, c),
            # which means that c is what you get when concatenating b and y
            [conso(a, b, x), conso(a, c, res), appendo(b, y, c) ]) 
    return fresh(_f)

list(run(
        lambda x: appendo(consify([1, 2]), consify([3,4]), x),
        ))

[{'x': [1, [2, [3, [4, []]]]]}]

In [122]:
list(run(
        lambda x, y: appendo(x, consify([1, 2]), y),
        stop=3))

[{'x': [], 'y': [1, [2, []]]},
 {'x': ['#_2', []], 'y': ['#_2', [1, [2, []]]]},
 {'x': ['#_2', ['#_5', []]], 'y': ['#_2', ['#_5', [1, [2, []]]]]}]

In [116]:
list(run(
        lambda x, y: appendo(x, y, consify(range(10))),
        stop=5))

[{'x': [], 'y': [0, [1, [2, [3, [4, [5, [6, [7, [8, [9, []]]]]]]]]]]},
 {'x': [0, []], 'y': [1, [2, [3, [4, [5, [6, [7, [8, [9, []]]]]]]]]]},
 {'x': [0, [1, []]], 'y': [2, [3, [4, [5, [6, [7, [8, [9, []]]]]]]]]},
 {'x': [0, [1, [2, []]]], 'y': [3, [4, [5, [6, [7, [8, [9, []]]]]]]]},
 {'x': [0, [1, [2, [3, []]]]], 'y': [4, [5, [6, [7, [8, [9, []]]]]]]}]